In [ ]:
# %cd /content
# !apt-get -y install -qq aria2

# !git clone -b v2.5 https://github.com/camenduru/text-generation-webui
# %cd /content/text-generation-webui
# !pip install -q -r requirements.txt

# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00001-of-00002.safetensors -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o model-00001-of-00002.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00002-of-00002.safetensors -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o model-00002-of-00002.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/model.safetensors.index.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o model.safetensors.index.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/special_tokens_map.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o special_tokens_map.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/tokenizer.model -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o tokenizer.model
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/tokenizer_config.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o tokenizer_config.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/config.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o config.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/generation_config.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o generation_config.json

# !echo "dark_theme: true" > /content/settings.yaml
# !echo "chat_style: wpp" >> /content/settings.yaml

# %cd /content/text-generation-webui
# !python server.py --share --settings /content/settings.yaml --model /content/text-generation-webui/models/Llama-2-7b-chat-hf

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install virtualenv
!pip install transformers
!pip install sentencepiece
!pip install langchain
!pip install sentence_transformers
!pip install bitsandbytes-cuda110
!pip install accelerate
!pip install bitsandbytes
# !pip3 install torch torchvision torchaudio
# !pip install packaging
# !pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.4 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, AutoModel, BitsAndBytesConfig
import torch
import sys
from torch import cuda, bfloat16
import transformers
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFacePipeline
# sys.path.append("/content/drive/MyDrive/CoLab/Llama2/bin/python3.10/site-packages")

In [4]:
# very important part, we quantilize Llama model by these codes
quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [5]:
import accelerate
import bitsandbytes

tokenzier=AutoTokenizer.from_pretrained("/content/drive/MyDrive/CoLab/LLama7b")  # daryl149/llama-2-7b-chat-hf
model=LlamaForCausalLM.from_pretrained("/content/drive/MyDrive/CoLab/LLama7b",quantization_config=quant_config,device_map="auto")
# model=AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/CoLab/LLama7b", trust_remote_code=False, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# !mkdir "/content/drive/MyDrive/CoLab/LLama7b"
# tokenzier.save_pretrained("/content/drive/MyDrive/CoLab/LLama7b")
# model.save_pretrained("/content/drive/MyDrive/CoLab/LLama7b")

In [6]:
# load on text splitter and initialize text loading
loader=TextLoader(r"/content/drive/MyDrive/CoLab/Test_data/prj_test.txt")
text=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=40400, chunk_overlap=0)
docs=text_splitter.split_documents(text)

In [ ]:
# this is another embedding method, dont know whether comptable with current docs format
# embed_tokenizer=AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
# embed_model=AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
model_trans="sentence-transformers/all-MiniLM-L6-v2"
model_kwargs={"device": 0}
embedding=HuggingFaceEmbeddings(model_name=model_trans, model_kwargs=model_kwargs)

In [7]:
docs_extend=[val.page_content for val in docs]

In [ ]:
#---------new added line to test text and embedding function comptability--------------
# print(type(docs), type(text[0].page_content))
# query_result=embedding.embed_documents(docs)
# len(query_result)

In [ ]:
text[0]

In [ ]:
device=f"cuda: {cuda.current_device()}"

tokenzier.convert_ids_to_tokens([9427])

['▁fish']

Do we need a Promopt template?

In [8]:
def risk_template(words: str):
    return f"""help me to identify and extract risks in the {words}? please mark it out and specify the type.
    Also pick keywords to label what ENTIRE sentence talked about from given tags:
    [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance].
    If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show risks, type of risks, keyword and tags appeared in sentence separately.
    Attention, you can only predict the output in above given criteria and no more than 40 words.No need to specific mark out on every sentence

    Answer example: "
    risks: Construction cost overrun, Ship collision and grounding \n

    risk_type : Financial, Technical \n

    keywords : Risk Management \n

    tags : Environment Science, Environment Studies, Finance" \n
    Answer no more than 20 words.
"""
query="help me to identify and extract risks in the given documents? please mark it out and specify the type."

In [9]:
!nvidia-smi

Wed Sep 13 05:02:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    41W / 300W |   5228MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
prefix_llm=transformers.pipeline(
    model=model, tokenizer=tokenzier,
    return_full_text=True,
    task="text-generation",temperature=0,
    top_p=0.15, top_k=15,
    max_new_tokens=318, repetition_penalty=1.2
)

In [11]:
llm=HuggingFacePipeline(pipeline=prefix_llm, model_kwargs={'temperature':0})
chain=load_qa_chain(llm=llm)

In [12]:
res_test=chain.run(input_documents=docs, question=risk_template("words"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `15` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maxi

In [ ]:
total_len=sum([len(docs[i].page_content) for i in range(len(docs))])

In [ ]:
print(res_test, "\n", total_len)



risk: Time delays, budget overruns, safety hazards 


risk_type: Operational, Financial 

keywords: Project Management, Safety Measures 

tags: Engineering Civil, Transportation 
 12631
